# Getting Geographical Targeting Insights Using Census Data from CensusReporter's API 
## By Tyler Chambers
### Created for APRD6432: Digital Advertising

## Project Summary

We have been tasked with helping a Business decide what metropolitan area it should build its next retail location in. Their most recent marketing research report shows that their product is most popular with people aged 18-34, whom have a yearly household income of $50,000-75,000 a year, and whom are Hispanic in background. We can help advise this Business on where they might want to relocate by using census data to find the metropolitan areas with the highest numbers of people with this demographic profile. Using a file with 382 US metropolitan codes, we will call CensusReporter's API to get and analyze this data. 

# Setting up our environment for the API and data collection

In [12]:
#importing packages for later use
import pandas as pd
import requests
import json
from time import sleep

#Importing the CSV file, you can download it also from my github 
Filename = 'msas.csv'
#Putting the csv data in a dataframe msas
msas = pd.read_csv(Filename)

#Making a single Table IDs for the two requests.
#For some reason the Hispanic population data did not like to be pulled at same time, so we will handle it separately
tableids = ['B01001','B19001'] 

#Reformatting list to string to properly feed into the API request URL
tableidstring = ','.join(tableids)

#Pulling the geoids from our dataframe into a list
geoid = msas['CBSA']
geoid = geoid.tolist()
print(geoid[0:5])

#Creating request URL
##requesturl = 'http://api.censusreporter.org/1.0/data/show/latest?table_ids=%s&geo_ids=31000US%s' % (tableidstring, geoid)

#Creating DataFrame to hold Census Data
metrodata = pd.read_csv(Filename)
metrodata['Age 18-34'] = 1
metrodata['Age Percent of Max'] = float(1)
metrodata['Percentage of Population that is Hispanic'] = 1
metrodata['Hispanic Percent of Max'] = float(1)
metrodata['HH Income 50k-74k'] = 1
metrodata['HH Income Percent of Max'] = float(1)
metrodata['Average Percentage'] = float(0)

#Creating lists to hold data from for loop
totalagelist=[0]
hispanicpopulationlist=[0]
totalHHlist=[0]

[10180, 10420, 10500, 10540, 10580]


## Creating the two functions that will run our API request

In [13]:
#Developing for loop to run our API request at scale for age groups
#Area responds to actual geocode, needed to wrap str() around it to make it a string value in the individual data requests
#Sleep was added to not overload the APIs servers
#To extract data from JSON, I used a series of get commands to fetch and hold the next dictionary key
# M# and F# correspond to specific population values for male and female, had to snag several individual data points and sum them
def getdataageHH(geoid):
    for area in geoid:
        requesturl = 'http://api.censusreporter.org/1.0/data/show/latest?table_ids=%s&geo_ids=31000US%s' % (tableidstring, area)
        loadedjson = requests.get(requesturl)
        parsedjson = json.loads(loadedjson.text)
        M18 = parsedjson.get('data', {}).get('31000US' + str(area), {}).get('B01001', {}).get('estimate', {}).get('B01001007')
        M20 = parsedjson.get('data', {}).get('31000US' + str(area), {}).get('B01001', {}).get('estimate', {}).get('B01001008')
        M21 = parsedjson.get('data', {}).get('31000US' + str(area), {}).get('B01001', {}).get('estimate', {}).get('B01001009')
        M24 = parsedjson.get('data', {}).get('31000US' + str(area), {}).get('B01001', {}).get('estimate', {}).get('B01001010')
        M29 = parsedjson.get('data', {}).get('31000US' + str(area), {}).get('B01001', {}).get('estimate', {}).get('B01001011')
        M34 = parsedjson.get('data', {}).get('31000US' + str(area), {}).get('B01001', {}).get('estimate', {}).get('B01001012')
        F18 = parsedjson.get('data', {}).get('31000US' + str(area), {}).get('B01001', {}).get('estimate', {}).get('B01001031')
        F20 = parsedjson.get('data', {}).get('31000US' + str(area), {}).get('B01001', {}).get('estimate', {}).get('B01001032')
        F21 = parsedjson.get('data', {}).get('31000US' + str(area), {}).get('B01001', {}).get('estimate', {}).get('B01001033')
        F24 = parsedjson.get('data', {}).get('31000US' + str(area), {}).get('B01001', {}).get('estimate', {}).get('B01001034')
        F29 = parsedjson.get('data', {}).get('31000US' + str(area), {}).get('B01001', {}).get('estimate', {}).get('B01001035')
        F34 = parsedjson.get('data', {}).get('31000US' + str(area), {}).get('B01001', {}).get('estimate', {}).get('B01001036')
        totalage = M18+M20+M21+M24+M29+M34+F18+F20+F21+F24+F29+F34
        totalagelist.append(totalage)
        HH50 = parsedjson.get('data', {}).get('31000US' + str(area), {}).get('B19001', {}).get('estimate', {}).get('B19001011')
        HH75 = parsedjson.get('data', {}).get('31000US' + str(area), {}).get('B19001', {}).get('estimate', {}).get('B19001012')
        totalHH = HH50 + HH75
        totalHHlist.append(totalHH)
        sleep(1)

#Very similar function to the one above, except for only the Hispanic population data       
def getdatahisp(geoid):
    for area in geoid:
        requesturl = 'https://api.censusreporter.org/1.0/data/show/latest?table_ids=B03002&geo_ids=31000US%s' % (area)
        loadedjson = requests.get(requesturl)
        parsedjson = json.loads(loadedjson.text)
        hispanic = parsedjson.get('data', {}).get('31000US' + str(area), {}).get('B03002', {}).get('estimate', {}).get('B03002012')
        hispanicpopulationlist.append(hispanic)
        sleep(1)

## Running our two functions to call the API data

In [14]:
#Running my data subsets against the API    
getdataageHH(geoid)
getdatahisp(geoid)

## Creating and Analyzing our Dataframe using Pandas

In [15]:
#Fix lists by removing 0 placeholder entry
totalagelist.remove(0)
hispanicpopulationlist.remove(0)
totalHHlist.remove(0)

#Putting lists into the dataframe
metrodata['Age 18-34'] = metrodata['Age 18-34'] * totalagelist
metrodata['Percentage of Population that is Hispanic'] = metrodata['Percentage of Population that is Hispanic'] * hispanicpopulationlist
metrodata['HH Income 50k-74k'] = metrodata['HH Income 50k-74k'] * totalHHlist

#Getting descriptives for the dataframe
print('\nDescriptives for API data')
print('--------------------------')
print(metrodata['Age 18-34'].describe())
print(metrodata['Percentage of Population that is Hispanic'].describe())
print(metrodata['HH Income 50k-74k'].describe())

#Storing maximum values for each 
maxage = metrodata['Age 18-34'].max()
maxhispanic = metrodata['Percentage of Population that is Hispanic'].max()
maxHH = metrodata['HH Income 50k-74k'].max()

#Filling in percentage column rows
metrodata['Age Percent of Max'] = metrodata['Age 18-34']/maxage
metrodata['Hispanic Percent of Max'] = metrodata['Percentage of Population that is Hispanic']/maxhispanic
metrodata['HH Income Percent of Max'] = metrodata['HH Income 50k-74k']/maxHH
metrodata['Average Percentage'] = (metrodata['Age Percent of Max']+metrodata['Hispanic Percent of Max']+metrodata['HH Income Percent of Max'])/3



Descriptives for API data
--------------------------
count    3.820000e+02
mean     1.734517e+05
std      3.983411e+05
min      1.121400e+04
25%      3.295125e+04
50%      6.105500e+04
75%      1.324970e+05
max      4.806939e+06
Name: Age 18-34, dtype: float64
count    3.820000e+02
mean     1.435339e+05
std      5.069093e+05
min      9.170000e+02
25%      6.926000e+03
50%      2.270050e+04
75%      8.114275e+04
max      6.031492e+06
Name: Percentage of Population that is Hispanic, dtype: float64
count    3.820000e+02
mean     4.665724e+04
std      9.359307e+04
min      4.042000e+03
25%      9.957250e+03
50%      1.673200e+04
75%      3.997875e+04
max      1.038188e+06
Name: HH Income 50k-74k, dtype: float64


## Finding Markets that Best Fit each of our Business Criteria

In [16]:
print('\n#Q1: Highest Population of People with a Household Income 50k-74k')
print('=================================================')
print(metrodata['NAME'][metrodata['HH Income 50k-74k'].idxmax()])
print(maxHH)

print('\n#Q2: Highest Population of 18-34 Year-olds')
print('===========================================')
print(metrodata['NAME'][metrodata['Age 18-34'].idxmax()])
print(maxage)

print('\n#Q3: Highest Population of peoples of Hispanic Background')
print('==========================================================')
print(metrodata['NAME'][metrodata['Percentage of Population that is Hispanic'].idxmax()])
print(maxHH)

print('\n#Q4: Highest Percentage of Maximum Values Across All Categories')
print('=========================================================')
print(metrodata['NAME'][metrodata['Average Percentage'].idxmax()])
print(metrodata['Average Percentage'].max())


#Q1: Highest Population of People with a Household Income 50k-74k
New York-Newark-Jersey City, NY-NJ-PA
1038188.0

#Q2: Highest Population of 18-34 Year-olds
New York-Newark-Jersey City, NY-NJ-PA
4806939.0

#Q3: Highest Population of peoples of Hispanic Background
Los Angeles-Long Beach-Anaheim, CA
1038188.0

#Q4: Highest Percentage of Maximum Values Across All Categories
New York-Newark-Jersey City, NY-NJ-PA
0.9426843308421864
